In [5]:
# !pip install ultralytics onnxruntime numpy opencv

In [1]:
import os
import time
import cv2 
import numpy as np
import onnxruntime as ort
from ultralytics import YOLO

In [2]:
MODEL_PT_PATH = '../model/best.pt'

DATASET_YAML_PATH = '../bottle_cap_dataset/dataset.yaml'

MODEL_ONNX_PATH = '../model/best_int8.onnx'

In [ ]:
model = YOLO(MODEL_PT_PATH)

try:
    model.export(
        format='onnx',
        imgsz=640,
        device='cpu',
        dynamic=True,
        simplify=True,
        opset=12,
    )
    exported_original_path = MODEL_PT_PATH.replace('.pt', '.onnx')
    os.rename(exported_original_path, MODEL_ONNX_PATH)
    
    print(f"\nSuccessfully exported and saved to: {MODEL_ONNX_PATH}")

except Exception as e:
    print(f"\nAn error occurred during export: {e}")

Ultralytics 8.3.228  Python-3.10.19 torch-2.2.2+cu121 CPU (AMD Ryzen 9 5900HS with Radeon Graphics)
WARNING half=True only compatible with GPU export, i.e. use device=0, setting half=False.
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '..\model\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.2 MB)

ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success  7.4s, saved as '..\model\best.onnx' (10.1 MB)

Export complete (8.9s)
Results saved to C:\Users\randy\Development\Computer Vision Projects\bottle_cap_detection\model
Predict:         yolo predict task=detect model=..\model\best.onnx imgsz=640  
Validate:        yolo val task=detect model=..\model\best.onnx imgsz=640 data=bottle_cap_dataset/dataset.yaml  
Visualize:       https://netron.app

An error occurred during export: [WinError 183] Cannot create a file when that file already exis

In [14]:
try:
    session = ort.InferenceSession(MODEL_ONNX_PATH, providers=['CPUExecutionProvider']) 

    input_name = session.get_inputs()[0].name
    input_shape = session.get_inputs()[0].shape  
    input_height = 640
    input_width = 640
    
    print(f"Model Input Shape: {input_shape}")

    print("Loading and pre-processing test image...")
    test_image_path = "../bottle_cap_dataset/images/val/raw-250110_dc_s001_b2_3.jpg" 
    
    if not os.path.exists(test_image_path):
        print(f"ERROR: Test image not found at {test_image_path}")
    else:
        image = cv2.imread(test_image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(image_rgb, (input_width, input_height))
        
        preprocessed_image = (resized / 255.0).transpose(2, 0, 1).astype(np.float32)
        
        # Add a batch dimension (B, C, H, W)
        input_tensor = np.expand_dims(preprocessed_image, axis=0)

        print("Running warm-up (20 inferences)")
        for _ in range(20):
            session.run(None, {input_name: input_tensor})

        num_runs = 100
        print(f"Running benchmark ({num_runs} inferences)")
        
        start_time = time.time()
        for _ in range(num_runs):
            session.run(None, {input_name: input_tensor})
        end_time = time.time()

        total_time = end_time - start_time
        avg_time_ms = (total_time / num_runs) * 1000
        fps = 1000 / avg_time_ms

        print("\nONNX Runtime Benchmark Results")
        print(f"Total time for {num_runs} inferences: {total_time:.2f} seconds")
        print(f"Average inference time: {avg_time_ms:.2f} ms")
        print(f"Frames Per Second (FPS): {fps:.2f}")

except Exception as e:
    print(f"Error during ONNX benchmark: {e}")

Model Input Shape: ['batch', 3, 'height', 'width']
Loading and pre-processing test image...
Running warm-up (20 inferences)
Running benchmark (100 inferences)

ONNX Runtime Benchmark Results
Total time for 100 inferences: 17.49 seconds
Average inference time: 174.88 ms
Frames Per Second (FPS): 5.72


In [9]:
import onnx
from onnx import TensorProto

def check_quantization(model_path):
    print(f"Inspecting model: {model_path}")
    
    model = onnx.load(model_path)
    graph = model.graph

    total_weights = len(graph.initializer)
    int8_weights = 0
    float_weights = 0
    
    print(f"Checking {total_weights} Weight Tensors")
    
    for tensor in graph.initializer:
        if tensor.data_type == TensorProto.INT8:
            int8_weights += 1
        elif tensor.data_type == TensorProto.UINT8:
            int8_weights += 1
        elif tensor.data_type == TensorProto.FLOAT:
            float_weights += 1

    print(f"INT8/UINT8 Weights: {int8_weights}")
    print(f"FLOAT32 Weights:    {float_weights}")

    q_nodes = [n for n in graph.node if n.op_type == "QuantizeLinear"]
    dq_nodes = [n for n in graph.node if n.op_type == "DequantizeLinear"]

    print(f"\nChecking Architecture (QDQ Nodes)")
    print(f"QuantizeLinear Nodes:   {len(q_nodes)}")
    print(f"DequantizeLinear Nodes: {len(dq_nodes)}")
    
    if int8_weights > 0 and len(q_nodes) > 0:
        print("\nThis model contains quantized INT8 weights and operators.")
    else:
        print("\nThis model appears to be FP32 (Unquantized).")

In [10]:
print("Checking FP32 model")
check_quantization("../model/best_fp32.onnx")

Checking FP32 model
Inspecting model: ../model/best_fp32.onnx
Checking 199 Weight Tensors
INT8/UINT8 Weights: 0
FLOAT32 Weights:    185

Checking Architecture (QDQ Nodes)
QuantizeLinear Nodes:   0
DequantizeLinear Nodes: 0

This model appears to be FP32 (Unquantized).


In [11]:
print("Checking INT8 model")
check_quantization("../model/best_int8.onnx")

Checking INT8 model
Inspecting model: ../model/best_int8.onnx
Checking 897 Weight Tensors
INT8/UINT8 Weights: 350
FLOAT32 Weights:    359

Checking Architecture (QDQ Nodes)
QuantizeLinear Nodes:   174
DequantizeLinear Nodes: 349

This model contains quantized INT8 weights and operators.
